In [185]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn import preprocessing
from sklearn import tree
from typing import List
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [186]:
def rules(n):
    totalminggu = n[0]
    totaldonor = n[1]
    totalminggu_now = n[2]
    totaldonor_now = n[3]
    if totaldonor <= 1:
        return 0.0
    
    if totalminggu/totaldonor < 16:
        return 1.0
    
    if totaldonor_now <= 1:
    # if totaldonor_now == 0:
        return 0.0

    if totalminggu_now/totaldonor_now < 16:
        return 1.0
    
    return 0.0


In [187]:

import random
import math


def generateData(n, with_label=True):
    data = []
    for i in range(0, n):
        minggu = random.randint(1, 392)

        totaldonor = 1 if math.floor(
            minggu/8) == 0 else random.randint(1, math.floor(minggu/8))
        totalminggu_now = random.randint(0, 27)
        totaldonor_now = random.randint(0,int(min(totaldonor-1,totalminggu_now/8))) # total donor now tidak mungkin lebih banyak daripada totaldonor
        if totalminggu_now == 0 or totaldonor_now == 0:
            totalminggu_now = 0
            totaldonor_now = 0

        d = [minggu, totaldonor, totalminggu_now, totaldonor_now]
        if with_label:
            data.append(d + [rules(d)])
        else:
            data.append(d)
    return data


data = generateData(1000)

real_df = pd.DataFrame(data, columns=["Jumlah Minggu Kesuluruhan", "Total Donor Keseluruhan",
                       "Jumlah Minggu Tahun Ini", "Total Donor Tahun Ini", "aktif"])
real_df.tail()


,Jumlah Minggu Kesuluruhan,Total Donor Keseluruhan,Jumlah Minggu Tahun Ini,Total Donor Tahun Ini,aktif
995,58,1,0,0,0.0
996,196,9,0,0,0.0
997,343,12,0,0,0.0
998,190,16,20,1,1.0
999,221,6,0,0,0.0


In [188]:
scalerX = MinMaxScaler()


def preprocessing(df:pd.DataFrame,categorical_columns:List[str],scaling:bool, unused_columns:List[str], label:str):
  df.dropna() # drop row bila salah 1 atau lebih ada NaN

  if len(categorical_columns) > 0:
    df = pd.get_dummies(data=df,columns=categorical_columns)
  
  # harus displit dulu antara features dan label, karena bila di minmax scalling dulu
  # nama kolom akan hilang, berubah menjadi array 2d
  unused_columns.append(label) # hapus kolom data column dan juga label
  X = df.drop(columns=unused_columns) 
  Y = df[label]
  Y = np.array(Y).reshape((-1,1))
  if scaling:
    global scalerX
    scalerX.fit(X.to_numpy())
    print("X")
    print(X)
    X = scalerX.transform(X.to_numpy())

    scaler = MinMaxScaler()
    scaler.fit(Y)
    Y = scaler.transform(Y)


  X = np.array(X)
  Y = np.array(Y).reshape((-1,1))
  
  return X, Y


In [189]:
real_df.head()

,Jumlah Minggu Kesuluruhan,Total Donor Keseluruhan,Jumlah Minggu Tahun Ini,Total Donor Tahun Ini,aktif
0,30,1,0,0,0.0
1,215,3,13,1,0.0
2,61,3,0,0,0.0
3,47,2,15,1,0.0
4,157,2,0,0,0.0


In [190]:
x_train, y_train = preprocessing(real_df,
                                                 scaling=False,
                                                 categorical_columns=[],
                                                  unused_columns=[],
                                                 label="aktif")

In [191]:
from sklearn import tree
model = tree.DecisionTreeClassifier()
model.fit(x_train, y_train)

DecisionTreeClassifier()

In [192]:
scaled_x_train, scaled_y_train = preprocessing(real_df,
                                                 scaling=True,
                                                 categorical_columns=[],
                                                 unused_columns=[],
                                                 label="aktif")

X
     Jumlah Minggu Kesuluruhan  Total Donor Keseluruhan  \
0                           30                        1   
1                          215                        3   
2                           61                        3   
3                           47                        2   
4                          157                        2   
..                         ...                      ...   
995                         58                        1   
996                        196                        9   
997                        343                       12   
998                        190                       16   
999                        221                        6   

     Jumlah Minggu Tahun Ini  Total Donor Tahun Ini  
0                          0                      0  
1                         13                      1  
2                          0                      0  
3                         15                      1  
4                  

In [193]:

# tranfusion_arr = tranfusion_df.to_numpy()
real_case_arr = generateData(100,False)

# tranfusion_arr =np.array(list(map(toWeek,tranfusion_arr)))
scaled_real_case_arr = scalerX.transform(real_case_arr)

In [194]:
scaled_y_train

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],

In [195]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(activation="relu",solver="lbfgs",shuffle=True)
mlp.fit(scaled_x_train, scaled_y_train.ravel())

C:\Users\c1419\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(solver='lbfgs')

In [196]:
real_case_arr[0:5]

[[354, 34, 0, 0],
 [324, 14, 0, 0],
 [196, 2, 0, 0],
 [206, 4, 0, 0],
 [292, 14, 23, 1]]

In [197]:
expectValue = list(map(lambda x: rules(x), real_case_arr))
expectValue[0:5]

[1.0, 0.0, 0.0, 0.0, 0.0]

expectValue digunakan untuk menvalidasi output dari model MLP dan Decision Tree  
Berdasarkan https://www.halodoc.com/artikel/berapa-kali-boleh-donor-darah-dalam-setahun dikatakan aktif(pria) bila interval waktu donor 8-14 minggu


In [198]:
MLPresult = mlp.predict(scaled_real_case_arr)

real_case_df = pd.DataFrame(real_case_arr,columns=["Jumlah Minggu Kesuluruhan", "Total Donor Keseluruhan", "Jumlah Minggu Sekarang", "Total Donor Sekarang"])
real_case_df["Predicted Value (MLP)"] = MLPresult
# real_case_df["Predicted Value (DT)"] = DTresult
real_case_df["Expected Value"] = expectValue

In [199]:
from IPython.display import display, HTML
# tranfusion_df
display(HTML("<h2>Hasil  Decision Tree (DT)</h2><p>source dataset: <a href='https://www.kaggle.com/datasets/ninalabiba/blood-transfusion-dataset?resource=download'>kaggle</a></p>"))
display(HTML(f"<div style='overflow-y:auto;max-height:200px'>{real_case_df.to_html()}</div>"))

,Jumlah Minggu Kesuluruhan,Total Donor Keseluruhan,Jumlah Minggu Sekarang,Total Donor Sekarang,Predicted Value (MLP),Expected Value
0,354,34,0,0,1.0,1.0
1,324,14,0,0,0.0,0.0
2,196,2,0,0,0.0,0.0
3,206,4,0,0,0.0,0.0
4,292,14,23,1,0.0,0.0
5,160,13,14,1,1.0,1.0
6,88,8,27,2,1.0,1.0
7,341,40,9,1,1.0,1.0
8,185,22,21,2,1.0,1.0
9,109,11,0,0,1.0,1.0


In [200]:
# Modified input
def modified(data,no_label=False):
    modified_data = []
    for i, d in enumerate(data):
        if d[1] == 0:
            interval_all = 0
        else:
            interval_all = d[0]/d[1]

        if d[3] == 0:
            interval_now = 0
        else:
            interval_now = d[2]/d[3]

        if no_label:
            modified_data.append([interval_all,interval_now,d[1],d[3]]) # kedua input digabung
        else:
            modified_data.append([interval_all,interval_now,d[1],d[3],d[4]]) # kedua input digabung
    return modified_data
modified_data = modified(data)
modified_data
modified_df = pd.DataFrame(modified_data,columns=["Mean Interval keseluruhan","Mean Interval terbaru","Total Donor keseluruhan","Total Donor terbaru","aktif"])
modified_df.head()

,Mean Interval keseluruhan,Mean Interval terbaru,Total Donor keseluruhan,Total Donor terbaru,aktif
0,30.000000,0.0,1,0,0.0
1,71.666667,13.0,3,1,0.0
2,20.333333,0.0,3,0,0.0
3,23.500000,15.0,2,1,0.0
4,78.500000,0.0,2,0,0.0


In [201]:
modifiedDT = tree.DecisionTreeClassifier()
modified_x_train, modified_y_train = preprocessing(modified_df, scaling=False,
                                                 categorical_columns=[],
                                                  unused_columns=[],
                                                 label="aktif")
modifiedDT.fit(modified_x_train, modified_y_train)
real_case_arr_modified = modified(real_case_arr,no_label=True)
modifiedDTresult = modifiedDT.predict(real_case_arr_modified) 

In [202]:
real_case_df["Predicted Value (modified DT)"] = modifiedDTresult

display(HTML("<h2>Hasil Perbandingan Deep Learning (MLP) dan Decision Tree (DT) dan Modified Decision Tree (DT)</h2>"))
display(HTML(f"<div style='overflow-y:auto;max-height:200px'>{real_case_df.to_html()}</div>"))

,Jumlah Minggu Kesuluruhan,Total Donor Keseluruhan,Jumlah Minggu Sekarang,Total Donor Sekarang,Predicted Value (MLP),Expected Value,Predicted Value (modified DT)
0,354,34,0,0,1.0,1.0,1.0
1,324,14,0,0,0.0,0.0,0.0
2,196,2,0,0,0.0,0.0,0.0
3,206,4,0,0,0.0,0.0,0.0
4,292,14,23,1,0.0,0.0,0.0
5,160,13,14,1,1.0,1.0,1.0
6,88,8,27,2,1.0,1.0,1.0
7,341,40,9,1,1.0,1.0,1.0
8,185,22,21,2,1.0,1.0,1.0
9,109,11,0,0,1.0,1.0,1.0


In [203]:
# fig = plt.figure(figsize=(25,20))
# _ = tree.plot_tree(model, 
#                    feature_names=["Donor 1 tahun terakhir","Jumlah hari terakhir donor", "frekuensi donor per tahun"],  
#                    class_names=["aktif"],
#                    filled=True)

import graphviz
# DOT data
dot_data = tree.export_graphviz(modifiedDT, out_file=None, 
                                feature_names=["Mean Interval keseluruhan","Mean Interval terbaru","Total Donor keseluruhan","Total Donor terbaru"],  
                                class_names=["non Aktif","Aktif"],
                                filled=True)

# Draw graph
graph = graphviz.Source(dot_data, format="png") 
graph.render("decision_tree_graphivz")

'decision_tree_graphivz.png'

In [204]:
import pickle
pickle.dump(modifiedDT, open("model", 'wb'))